 
</div>
 
 
<div style="padding: 10px; background-color: #960018; margin: 0 auto; color: #e7d954; font-family: newtimeroman; font-size: 150%; text-align: center; border-radius: 15px; overflow: hidden; font-weight: 1000; width: 100%;"> API   </div>




In [ ]:
from flask import Flask, request, jsonify
import pickle
import pandas as pd

In [ ]:
app = Flask(__name__)
excluded_columns = ['price_car', 'price_car_normalized','additions','additionsPlus'] 
feature_names = [col for col in df.columns if col not in excluded_columns]


In [ ]:
@app.route('/predict', methods=['POST'])
def predict():
    try:
        default_data = pd.DataFrame([[0] * len(feature_names)], columns=feature_names)
        data = request.get_json(force=True)
        app.logger.info(f"Received data: {data}")
        input_data = pd.DataFrame([data])
        features_used_during_training = input_data.columns
        input_data_selected = input_data[features_used_during_training]

        # Update the default_data with the user input
        default_data.update(input_data_selected)

        # Scale the 'Motor power' feature using the motor scaler
        default_data['motor_power'] = motorScaler.transform(default_data[['motor_power']])

        # Make predictions using the model
        predictions = rf_model.predict(default_data)

        # Inverse transform the normalized predictions
        original_scale_predictions = priceScaler.inverse_transform(predictions.reshape(-1, 1))

        # Log data before prediction
        app.logger.info(f"Predicting with data: {default_data}")

        # Return the unnormalized predictions
        return jsonify({'predicted_price': original_scale_predictions.tolist()})

    except Exception as e:
        app.logger.error(f"Error: {str(e)}")
        return jsonify({'error': f'Internal Server Error: {str(e)}'})

In [ ]:
if __name__ == '__main__':
    app.run(port=5000)